In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
from transformers import BertTokenizer, TFRobertaModel

In [ ]:
from google.colab import files


uploaded = files.upload()

Saving NLP.csv to NLP.csv


In [ ]:
import pandas as pd
import io

train_df = pd.read_csv(io.BytesIO(uploaded['NLP.csv']))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
def encoded_sentence(text):
    encoded_input = tokenizer(text)
    return encoded_input

In [ ]:
eg_output = encoded_sentence("I want to be a scientist")
eg_output

{'input_ids': [101, 146, 21528, 10114, 10347, 169, 59248, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def inputidstypes_sentence(premise,hypothesis):
    output = encoded_sentence(premise)
    input_ids = output['input_ids']
    input_types = output['token_type_ids']
    hypothesis_tokens = list(tokenizer.tokenize(hypothesis))
    hypothesis_tokens.append('[SEP]')
    hypothesis_ids = tokenizer.convert_tokens_to_ids(hypothesis_tokens)
    input_types.extend([1] * len(hypothesis_tokens))
    input_ids.extend(hypothesis_ids)
    return input_ids,input_types

In [ ]:
def model_input(premises,hypotheses):

    num_examples = len(hypotheses)
    print(num_examples)

    input_ids = list()
    input_types = list()

    for premise,hypothesis in zip(premises,hypotheses):
        input_id,input_type = inputidstypes_sentence(premise,hypothesis)
        input_ids.append(input_id)
        input_types.append(input_type)

    print(len(input_ids))
    print(len(input_types))
    input_words_ids =  tf.ragged.constant(input_ids)
    input_types_ids = tf.ragged.constant(input_types)
    input_mask = tf.ones_like(input_words_ids).to_tensor(shape=[num_examples, 259])
    print(input_words_ids.get_shape())

    inputs = {
      'input_word_ids': input_words_ids.to_tensor(shape=[num_examples, 259]),
      'input_mask': input_mask,
      'input_type_ids': input_types_ids.to_tensor(shape=[num_examples, 259])}

    return inputs


In [ ]:
xtrain = model_input(train_df.premise,train_df.hypothesis)


12120
12120
12120
(12120, None)


In [ ]:
max_len = 259

def build_model():
    bert_encoder = TFRobertaModel.from_pretrained("bert-base-multilingual-cased")
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")

    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5),loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    return model

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [ ]:
with strategy.scope():
    model = build_model()
    model.summary()

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['bert.encoder.layer.2.intermediate.dense.weight', 'bert.encoder.layer.10.output.dense.bias', 'bert.encoder.layer.11.output.dense.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.output.LayerNorm.weight', 'bert.encoder.layer.11.attention.self.query.bias', 'bert.encoder.layer.2.output.dense.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.self.key.bias', 'bert.pooler.dense.bias', 'bert.encoder.layer.1.attention.output.dense.bias', 'bert.encoder.layer.7.attention.output.dense.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.encoder.layer.3.output.dense.bias', 'bert.encoder.layer.4.attention.self.value.b

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 259)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 259)]                0         []                            
                                                                                                  
 input_type_ids (InputLayer  [(None, 259)]                0         []                            
 )                                                                                                
                                                                                            

In [ ]:
model.fit(xtrain, train_df.label.values, epochs = 10, verbose = 1, batch_size = 16, validation_split = 0.2)

Epoch 1/10
 22/606 [>.............................] - ETA: 8:48 - loss: 1.1003 - accuracy: 0.3693